In [159]:
# imports
import pandas as pd
import numpy as np

In [ ]:
# loading in the data

columns = [
    'variance',
    'skewness',
    'curtosis',
    'entropy',
    'label'
]

train_data = pd.read_csv('../bank/train.csv', names=columns,header=None)
test_data = pd.read_csv('../bank/test.csv', names=columns, header=None)

train_data

,variance,skewness,curtosis,entropy,label
0,3.848100,10.15390,-3.85610,-4.22280,0
1,4.004700,0.45937,1.36210,1.61810,0
2,-0.048008,-1.60370,8.47560,0.75558,0
3,-1.266700,2.81830,-2.42600,-1.88620,1
4,2.203400,5.99470,0.53009,0.84998,0
...,...,...,...,...,...
867,0.273310,4.87730,-4.91940,-5.81980,1
868,1.063700,3.69570,-4.15940,-1.93790,1
869,-1.242400,-1.71750,-0.52553,-0.21036,1
870,1.837300,6.12920,0.84027,0.55257,0


In [161]:
# need to convert the labels from [0,1] -> [-1,1]
# need to change the labels from [-1,1]
train_data.iloc[:, -1].replace(0, -1, inplace=True)
test_data.iloc[:, -1].replace(0, -1, inplace=True)

train_data

,variance,skewness,curtosis,entropy,label
0,3.848100,10.15390,-3.85610,-4.22280,-1
1,4.004700,0.45937,1.36210,1.61810,-1
2,-0.048008,-1.60370,8.47560,0.75558,-1
3,-1.266700,2.81830,-2.42600,-1.88620,1
4,2.203400,5.99470,0.53009,0.84998,-1
...,...,...,...,...,...
867,0.273310,4.87730,-4.91940,-5.81980,1
868,1.063700,3.69570,-4.15940,-1.93790,1
869,-1.242400,-1.71750,-0.52553,-0.21036,1
870,1.837300,6.12920,0.84027,0.55257,-1


In [162]:


# now need to split the data to features (columns) and labels (prediction)

x_train = train_data.drop('label', axis=1).values
y_train = train_data['label'].values

x_test = test_data.drop('label', axis=1).values
y_test = test_data['label'].values


# 2: SVM Primal Domain

In [204]:


#initializing variables
# max epochs 100
epoch = 100
#shuffle at the start of each epoch
c = [(100/873), (500/873), (700/873)]
#making a zero weight vector with the number of features
weights = np.zeros(x_train.shape[1])

bias = 0.0
N = len(x_train)
learning_rate = 0.01 #((rate)/(1+((rate/a)*t)))
a = 0.01

In [205]:
# the gradient is the hindge loss

def hinge_loss(w, b, x_i, y_i, c):
    loss_max = np.maximum(0, 1 - (y_i * (np.dot(x_i, w) + b)))
    reg_term = 0.5 * np.dot(w,w)
    return np

In [206]:
#implement SVM in primal domain with stochastic sub-gradient descent

def Primal_SVM_1(x_train, y_train, bias, N, epoch, weights, C, learning_rate, a):
    for t in range(epoch):
        # shuffle at the start of each epoch
        shuffle_idx = np.random.permutation(len(x_train))
        x_train_shuffled = x_train[shuffle_idx]
        y_train_shuffled = y_train[shuffle_idx]

        for feature, label in zip(x_train_shuffled, y_train_shuffled):
            #might need to add bias to wieghts and augment 1 in data for bias
            learning_rate = (learning_rate/(1+ (learning_rate/a)*t))
            predict = label * (np.dot(weights, feature) + bias)

            if(predict < 1):
                #miss classiffied example
                #might need to fix the first parenthesis
                weights = weights - (learning_rate * (-label*feature)) + (learning_rate * C * N * label * feature)
                bias = bias - (learning_rate * -label)
            else:
                # gradient is 0
            
                weights = (1 - learning_rate)*weights

    return weights, bias




In [207]:
# now primal with other learning rate

def Primal_SVM_2(x_train, y_train, bias, N, epoch, weights, C, learning_rate):
    for t in range(epoch):
        # shuffle at the start of each epoch
        shuffle_idx = np.random.permutation(len(x_train))
        x_train_shuffled = x_train[shuffle_idx]
        y_train_shuffled = y_train[shuffle_idx]

        for feature, label in zip(x_train_shuffled, y_train_shuffled):
            #might need to add bias to wieghts and augment 1 in data for bias
            learning_rate = (learning_rate/(1+t))
            predict = label * (np.dot(weights, feature) + bias)

            if(predict < 1):
                #miss classiffied example
                #might need to fix the first parenthesis
                weights = weights - (learning_rate * (-label*feature)) + (learning_rate * C * N * label * feature)
                bias = bias - (learning_rate * -label)
            else:
                # gradient is 0

                weights = (1 - learning_rate)*weights

    return weights, bias

In [208]:
def Error_val(feature, labels, weights, bias):
    predict = np.sign(np.dot(feature, weights) + bias)
    return np.mean(predict != labels)

In [209]:
# finding error
SVM_first_learning_rate = []
SVM_first_LR = []
for C in c: 
    w_first, b_first = Primal_SVM_1(x_train, y_train, bias, N, epoch, weights, C, learning_rate, a)
    
    train_err_first = Error_val(x_train, y_train, w_first, b_first)
    
    test_err_first = Error_val(x_test, y_test, w_first, b_first)
    
    SVM_first_learning_rate.append((w_first, b_first, train_err_first, test_err_first))
    SVM_first_LR.append((C, train_err_first, test_err_first))

In [210]:
print(SVM_first_learning_rate)

[(array([-14.19619787,  -8.26925256,  -9.52265237,   1.00832654]), 0.51136186748002, 0.06192660550458716, 0.094), (array([-44.03048856, -19.48204544, -24.67121278,  -3.64201134]), 0.5035534775371111, 0.04472477064220184, 0.064), (array([-66.30777738, -38.7073781 , -21.20035635, -31.77408991]), 0.4722403478225372, 0.06192660550458716, 0.082)]


In [211]:
print(SVM_first_LR)

[(0.1145475372279496, 0.06192660550458716, 0.094), (0.572737686139748, 0.04472477064220184, 0.064), (0.8018327605956472, 0.06192660550458716, 0.082)]


In [212]:
SVM_second_learning_rate = []
SVM_second_LR = []
for C in c:
    w_second, b_second = Primal_SVM_2(x_train, y_train, bias, N, epoch, weights, C, learning_rate)

    train_err_second = Error_val(x_train, y_train, w_second, b_second)

    test_err_second = Error_val(x_test, y_test, w_second, b_second)

    SVM_second_learning_rate.append((w_second, b_second, train_err_second, test_err_second))
    SVM_second_LR.append((C, train_err_second, test_err_second))

In [213]:
print(SVM_second_learning_rate)

[(array([-13.73707301,  -6.96722766, -10.47959213,   0.06077835]), 0.5203125011922409, 0.0963302752293578, 0.108), (array([-46.86138624, -29.53876689, -20.12985876, -16.83757671]), 0.6500099564343695, 0.05619266055045872, 0.074), (array([-25.79773735, -20.73117252, -19.67580833, -27.50361853]), 0.5500001526623968, 0.07568807339449542, 0.078)]


In [214]:
print(SVM_second_LR)

[(0.1145475372279496, 0.0963302752293578, 0.108), (0.572737686139748, 0.05619266055045872, 0.074), (0.8018327605956472, 0.07568807339449542, 0.078)]


In [215]:
# now compare the two
comparison_results = []
for i, C in enumerate(c):
    w_diff = np.linalg.norm(SVM_first_learning_rate[i][0] - SVM_second_learning_rate[i][0])
    b_diff = abs(SVM_first_learning_rate[i][1] - SVM_second_learning_rate[i][1])
    train_error_diff = abs(SVM_first_learning_rate[i][2] - SVM_second_learning_rate[i][2])
    test_error_diff = abs(SVM_first_learning_rate[i][3] - SVM_second_learning_rate[i][3])
    comparison_results.append({
        'C': C,
        'Diff in w': w_diff,
        'Diff in b': b_diff,
        'Diff in Train Error': train_error_diff,
        'Diff in Test Error': test_error_diff
    })

In [216]:
print("First Schedule Results:", SVM_first_learning_rate)
print("Alternate Schedule Results:", SVM_second_learning_rate)
print("Comparison Results:", comparison_results)

First Schedule Results: [(array([-14.19619787,  -8.26925256,  -9.52265237,   1.00832654]), 0.51136186748002, 0.06192660550458716, 0.094), (array([-44.03048856, -19.48204544, -24.67121278,  -3.64201134]), 0.5035534775371111, 0.04472477064220184, 0.064), (array([-66.30777738, -38.7073781 , -21.20035635, -31.77408991]), 0.4722403478225372, 0.06192660550458716, 0.082)]
Alternate Schedule Results: [(array([-13.73707301,  -6.96722766, -10.47959213,   0.06077835]), 0.5203125011922409, 0.0963302752293578, 0.108), (array([-46.86138624, -29.53876689, -20.12985876, -16.83757671]), 0.6500099564343695, 0.05619266055045872, 0.074), (array([-25.79773735, -20.73117252, -19.67580833, -27.50361853]), 0.5500001526623968, 0.07568807339449542, 0.078)]
Comparison Results: [{'C': 0.1145475372279496, 'Diff in w': 1.928638319943085, 'Diff in b': 0.008950633712220935, 'Diff in Train Error': 0.034403669724770644, 'Diff in Test Error': 0.013999999999999999}, {'C': 0.572737686139748, 'Diff in w': 17.43268395409465

In [217]:
print(comparison_results)

[{'C': 0.1145475372279496, 'Diff in w': 1.928638319943085, 'Diff in b': 0.008950633712220935, 'Diff in Train Error': 0.034403669724770644, 'Diff in Test Error': 0.013999999999999999}, {'C': 0.572737686139748, 'Diff in w': 17.43268395409465, 'Diff in b': 0.14645647889725844, 'Diff in Train Error': 0.011467889908256881, 'Diff in Test Error': 0.009999999999999995}, {'C': 0.8018327605956472, 'Diff in w': 44.55074053662368, 'Diff in b': 0.07775980483985956, 'Diff in Train Error': 0.013761467889908258, 'Diff in Test Error': 0.0040000000000000036}]


# 3 SVM in Duel Domain

In [16]:
from scipy.optimize import minimize

In [17]:
def Duel_svm(alpha, features, labels):

    feature_matrix = np.dot(features, features.T)  # Precompute the  matrix
    return 0.5 * np.sum(alpha[:, None] * alpha[None, :] * labels[:, None] * labels[None, :] * feature_matrix) - np.sum(alpha)

In [18]:


c = [(100/873), (500/873), (700/873)]


In [19]:
def constraint(alphas,y):
    return np.dot(alphas, y)

In [23]:
# need to use teh KKT conditions
def Duel_svn_recover(x, y, alphas):

    # if alphas are 0 then will not affect the weight vector

    weight_star = np.sum(alphas[:, None] * y[:, None] * x, axis=0)
    # makesure to det a thres for the non-zero ones
    support_vec = np.where(alphas > 1e-5)[0]


    if len(support_vec) == 0:
        return weight_star, np.nan


    bias_star = np.mean(y[support_vec] - np.dot(x[support_vec], weight_star))

    return weight_star, bias_star

In [26]:
svm_Duel = []
for C in c:
    constraints = ({'type': 'eq', 'fun': constraint, 'args': (y_train,)})
    inital_alpha = np.zeros(len(y_train))
    bounds = [(0, C) for _ in range(len(y_train))]
    result = minimize(Duel_svm, inital_alpha, args=(x_train, y_train),
                      method='SLSQP', bounds=bounds, constraints=constraints)

    optimal_alphas = result.x
    w, b = Duel_svn_recover(x_train, y_train, optimal_alphas)
    #print(w,b)
    svm_Duel.append((w,b,C))


C:\Users\Baile\anaconda3\Lib\site-packages\scipy\optimize\_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


[-0.94292638 -0.65149177 -0.73372188 -0.04102186] 1.074503324130715
[-1.56393891 -1.01405315 -1.18065195 -0.15651812] 1.512365468870149
[-2.04256002 -1.28069441 -1.51352246 -0.24905442] 1.8793843116087874


[0.1145475372279496, 0.572737686139748, 0.8018327605956472]


In [27]:
print(svm_Duel)

[(array([-0.94292638, -0.65149177, -0.73372188, -0.04102186]), 1.074503324130715, 0.1145475372279496), (array([-1.56393891, -1.01405315, -1.18065195, -0.15651812]), 1.512365468870149, 0.572737686139748), (array([-2.04256002, -1.28069441, -1.51352246, -0.24905442]), 1.8793843116087874, 0.8018327605956472)]


In [30]:
# for val in zip(svm_Duel):
#     print(f"w: {', '.join(map(str, w))}, b: {b}, C: {c}")

w: -2.0425600248469213, -1.2806944147568369, -1.5135224605170117, -0.2490544211622434, b: 1.8793843116087874, C: [0.1145475372279496, 0.572737686139748, 0.8018327605956472]
w: -2.0425600248469213, -1.2806944147568369, -1.5135224605170117, -0.2490544211622434, b: 1.8793843116087874, C: [0.1145475372279496, 0.572737686139748, 0.8018327605956472]
w: -2.0425600248469213, -1.2806944147568369, -1.5135224605170117, -0.2490544211622434, b: 1.8793843116087874, C: [0.1145475372279496, 0.572737686139748, 0.8018327605956472]


# Gaussian Kernel

In [143]:
def gaussian_Kernel(x1,x2, gam):

    return np.exp(((np.linalg.norm(x1 - x2)**2)/ gam))

    

In [144]:
gammas = [0.1, 0.5, 1, 5, 100]
c = [(100/873), (500/873), (700/873)]

In [146]:
def duel_kernel(alpha, features, labels, gamma):
    #use the kernel
    kernal = gaussian_Kernel(features, features, gamma)

    return 0.5 * np.sum(alpha[:, None] * alpha[None, :] * labels[:, None] * labels[None, :] * kernal) - np.sum(alpha)

In [148]:
def predict_with_gauss_train(train_x, y, test_x, alpha, bias, gamma):
    kernal = gaussian_Kernel(train_x, train_x, gamma)
    predict = np.sign(np.sum(kernal * alpha * y) + bias)
    return predict

In [156]:
def predict_with_gauss_test(train_x, y, test_x, alpha, bias, gamma):
 
    
    #to account for different test and train shapes
    num_test_samples = len(test_x)
    num_train_samples = len(train_x)

    predictions = np.zeros(num_test_samples)

    for i in range(num_test_samples):
        kernal = gaussian_Kernel(test_x[i], train_x, gamma)
        predictions[i] = np.sign(np.sum(kernal * alpha[:num_train_samples] * y[:num_train_samples, None]) + bias)

    return predictions



In [ ]:
result_kern = {}
support_result = {}
support_result_num={}
for gam in gammas:
    for C in c:
        constraints = ({'type': 'eq', 'fun': constraint, 'args': (y_train,)})
        bounds = [(0, C) for _ in range(len(y_train))]
        initial_alpha = np.zeros(len(y_train))
        result = minimize(duel_kernel, initial_alpha, args=(x_train, y_train, gam),
                          method='SLSQP', bounds=bounds, constraints=constraints)
        
        opt_alphas = result.x
      
        weights, bias = Duel_svn_recover(x_train, y_train, opt_alphas)
        train_predict = predict_with_gauss_train(x_train, y_train, x_train, opt_alphas, bias, gam)
        
        test_predict = predict_with_gauss_test(x_train, y_test, x_test, opt_alphas, bias, gam)
        
        training_err = np.mean(train_predict != y_train)
        test_err = np.mean(test_predict != y_test)
        
        result_kern[(gam, C)] = (training_err, test_err)
        
        support_result[(gam, C)] = opt_alphas
        support_result_num[(gam, C)] = len(opt_alphas)
        
        

C:\Users\Baile\AppData\Local\Temp\ipykernel_14860\3285923321.py:6: RuntimeWarning: overflow encountered in exp
  return np.exp(((np.linalg.norm(x1 - x2)**2)/ gam))


In [158]:
print(result_kern)

{(0.1, 0.1145475372279496): (0.5538990825688074, 1.0), (0.1, 0.572737686139748): (0.5538990825688074, 1.0), (0.1, 0.8018327605956472): (0.5538990825688074, 1.0), (0.5, 0.1145475372279496): (0.5538990825688074, 1.0), (0.5, 0.572737686139748): (0.5538990825688074, 1.0), (0.5, 0.8018327605956472): (0.5538990825688074, 1.0), (1, 0.1145475372279496): (0.5538990825688074, 1.0), (1, 0.572737686139748): (0.5538990825688074, 1.0), (1, 0.8018327605956472): (0.5538990825688074, 1.0), (5, 0.1145475372279496): (0.5538990825688074, 1.0), (5, 0.572737686139748): (0.5538990825688074, 1.0), (5, 0.8018327605956472): (0.5538990825688074, 1.0), (100, 0.1145475372279496): (0.5538990825688074, 0.91), (100, 0.572737686139748): (0.5538990825688074, 0.914), (100, 0.8018327605956472): (0.5538990825688074, 0.914)}


In [220]:
print(support_result)

{(0.1, 0.1145475372279496): array([0.09225464, 0.09225464, 0.09225464, 0.11454754, 0.09225464,
       0.09225464, 0.09225464, 0.09225464, 0.09225464, 0.09225464,
       0.11454754, 0.11454754, 0.09225464, 0.11454754, 0.11454754,
       0.11454754, 0.09225464, 0.09225464, 0.11454754, 0.11454754,
       0.11454754, 0.11454754, 0.11454754, 0.09225464, 0.09225464,
       0.11454754, 0.09225464, 0.09225464, 0.09225464, 0.09225464,
       0.09225464, 0.09225464, 0.11454754, 0.11454754, 0.09225464,
       0.09225464, 0.11454754, 0.11454754, 0.09225464, 0.09225464,
       0.11454754, 0.09225464, 0.09225464, 0.11454754, 0.11454754,
       0.11454754, 0.11454754, 0.11454754, 0.09225464, 0.09225464,
       0.09225464, 0.11454754, 0.11454754, 0.11454754, 0.09225464,
       0.09225464, 0.11454754, 0.09225464, 0.11454754, 0.11454754,
       0.11454754, 0.09225464, 0.11454754, 0.09225464, 0.09225464,
       0.09225464, 0.11454754, 0.09225464, 0.09225464, 0.11454754,
       0.09225464, 0.09225464, 0.1

In [218]:
# Output the results
for key, value in result_kern.items():
    print(f"Gamma: {key[0]}, C: {key[1]} => Train Error: {value[0]}, Test Error: {value[1]}")

Gamma: 0.1, C: 0.1145475372279496 => Train Error: 0.5538990825688074, Test Error: 1.0
Gamma: 0.1, C: 0.572737686139748 => Train Error: 0.5538990825688074, Test Error: 1.0
Gamma: 0.1, C: 0.8018327605956472 => Train Error: 0.5538990825688074, Test Error: 1.0
Gamma: 0.5, C: 0.1145475372279496 => Train Error: 0.5538990825688074, Test Error: 1.0
Gamma: 0.5, C: 0.572737686139748 => Train Error: 0.5538990825688074, Test Error: 1.0
Gamma: 0.5, C: 0.8018327605956472 => Train Error: 0.5538990825688074, Test Error: 1.0
Gamma: 1, C: 0.1145475372279496 => Train Error: 0.5538990825688074, Test Error: 1.0
Gamma: 1, C: 0.572737686139748 => Train Error: 0.5538990825688074, Test Error: 1.0
Gamma: 1, C: 0.8018327605956472 => Train Error: 0.5538990825688074, Test Error: 1.0
Gamma: 5, C: 0.1145475372279496 => Train Error: 0.5538990825688074, Test Error: 1.0
Gamma: 5, C: 0.572737686139748 => Train Error: 0.5538990825688074, Test Error: 1.0
Gamma: 5, C: 0.8018327605956472 => Train Error: 0.5538990825688074, 

In [221]:
# Output the results
for key, value in support_result.items():
    print(f"Gamma: {key[0]}, C: {key[1]} =>support Vectors: {value[0].size}")

Gamma: 0.1, C: 0.1145475372279496 =>support Vectors: 0.5538990825688074
Gamma: 0.1, C: 0.572737686139748 =>support Vectors: 0.5538990825688074
Gamma: 0.1, C: 0.8018327605956472 =>support Vectors: 0.5538990825688074
Gamma: 0.5, C: 0.1145475372279496 =>support Vectors: 0.5538990825688074
Gamma: 0.5, C: 0.572737686139748 =>support Vectors: 0.5538990825688074
Gamma: 0.5, C: 0.8018327605956472 =>support Vectors: 0.5538990825688074
Gamma: 1, C: 0.1145475372279496 =>support Vectors: 0.5538990825688074
Gamma: 1, C: 0.572737686139748 =>support Vectors: 0.5538990825688074
Gamma: 1, C: 0.8018327605956472 =>support Vectors: 0.5538990825688074
Gamma: 5, C: 0.1145475372279496 =>support Vectors: 0.5538990825688074
Gamma: 5, C: 0.572737686139748 =>support Vectors: 0.5538990825688074
Gamma: 5, C: 0.8018327605956472 =>support Vectors: 0.5538990825688074
Gamma: 100, C: 0.1145475372279496 =>support Vectors: 0.5538990825688074
Gamma: 100, C: 0.572737686139748 =>support Vectors: 0.5538990825688074
Gamma: 10

In [222]:
# Output the results
for key, value in support_result.items():
    if key[1] == 500/873:
        print(f"Gamma: {key[0]}, C: {key[1]} =>support Vectors: {value[0].size}")

Gamma: 0.1, C: 0.572737686139748 =>support Vectors: 0.5538990825688074
Gamma: 0.5, C: 0.572737686139748 =>support Vectors: 0.5538990825688074
Gamma: 1, C: 0.572737686139748 =>support Vectors: 0.5538990825688074
Gamma: 5, C: 0.572737686139748 =>support Vectors: 0.5538990825688074
Gamma: 100, C: 0.572737686139748 =>support Vectors: 0.5538990825688074
